In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "PPO2"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [4]:
folder = '/mnt/d/donkeycar_model/PPO2/'

In [5]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [8]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


starting DonkeyGym env


In [9]:
for i in range(1000):
    # preparation
    if i == 0:
        model = model = PPO("CnnPolicy", env, n_steps=500, verbose=2 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_PPO2-{(i+1)*2500}.npy", Log)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 500`, after every 7 untruncated mini-batches, there will be a truncated mini-batch of size 52
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=500 and n_envs=1)
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.6     |
|    ep_rew_mean     | -33.4    |
| time/              |          |
|    fps             | 35       |
|    iterations      | 1        |
|    time_elapsed    | 13       |
|    total_timesteps | 500      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 83.3        |
|    ep_rew_mean          | -6.25       |
| time/                   |             |
|    fps                  | 29          |
|    iterations           | 2           |
|    time_elapsed         | 33          |
|    total_timesteps      | 1000        |
| train/                  |             |
|    approx_kl            | 0.023145609 |
|    clip_fraction        | 0.479       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.84       |
|    explained_variance   | -0.00134    |
|    learning_rate        | 0.

/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/utils.py:573: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return th.as_tensor(obs, device=device)


testing 2500 : -103.60287907748804
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 28.2     |
|    ep_rew_mean     | -103     |
| time/              |          |
|    fps             | 15       |
|    iterations      | 1        |
|    time_elapsed    | 32       |
|    total_timesteps | 500      |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 24.3       |
|    ep_rew_mean          | -76.8      |
| time/                   |            |
|    fps                  | 13         |
|    iterations           | 2          |
|    time_elapsed         | 73         |
|    total_timesteps      | 1000       |
| train/                  |            |
|    approx_kl            | 0.43433112 |
|    clip_fraction        | 0.782      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.93      |
|    explained_variance   | 0.326      |
|    learn

KeyboardInterrupt: 

In [86]:
num = 60000
test_model = PPO.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    # print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

1343.4235813868684


In [10]:
env.close()